# Introduction

# Iniatial

1. Import Necessary Libraries


In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

2. Load the Datasets


In [151]:
all_leagues = pd.read_csv('clubs_n.csv')
players_df = pd.read_csv('/content/playerss.csv')
doctors_df = pd.read_csv('doctorsRa.csv')
coachs_df = pd.read_csv('cleaned_coaches (1).csv')
analyst_df = pd.read_csv('DataAnalyst_Modified.csv')

# 3. Data Preprocessing

In [152]:
# Standardize column names
def standardize_column_names(df):
    df.columns = df.columns.str.replace('possession(%)', 'possession(%)')
    df.columns = df.columns.str.replace('pass_accuracy(%)', 'pass_accuracy(%)')
    return df

all_leagues = standardize_column_names(all_leagues)

# Handle missing values
def handle_missing_values(df):
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cat_cols = df.select_dtypes(include=['object']).columns

    df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])
    return df

all_leagues = handle_missing_values(all_leagues)

# Remove duplicates
all_leagues = all_leagues.drop_duplicates()

#4. Data Understanding


In [153]:
# Drop the 'total_market_value' column
all_leagues = all_leagues.drop(columns=['total_market_value','coach_name','club_standardized_y','url','filename'])
# Impute missing values in clubs dataset
all_leagues['average_age'] = all_leagues['average_age'].fillna(all_leagues['average_age'].mean())
all_leagues['foreigners_percentage'] = all_leagues['foreigners_percentage'].fillna(all_leagues['foreigners_percentage'].mean())

# Verify the result
missing_club_data_after = all_leagues.isna().sum()
print("🔹 Missing Data in Clubs Dataset After Imputation:")
print(missing_club_data_after[missing_club_data_after > 0])

🔹 Missing Data in Clubs Dataset After Imputation:
Series([], dtype: int64)


In [154]:
# Check for missing values
print(players_df.isnull().sum().sum())

0


In [155]:
# Step 1: Identify the percentage of missing values per column
missing_percentage = players_df.isnull().mean() * 100

# Step 2: Drop columns with more than 50% missing values
cols_to_drop = missing_percentage[missing_percentage > 50].index
players_df.drop(columns=cols_to_drop, inplace=True)

# Step 3: Separate numerical and categorical columns
numerical_cols = players_df.select_dtypes(include='number').columns
categorical_cols = players_df.select_dtypes(include='object').columns

# Step 4: Impute numerical columns with median
for col in numerical_cols:
    if players_df[col].isnull().sum() > 0:
        players_df[col].fillna(players_df[col].median(), inplace=True)

# Step 5: Impute categorical columns with mode
for col in categorical_cols:
    if players_df[col].isnull().sum() > 0:
        players_df[col].fillna(players_df[col].mode()[0], inplace=True)

# Step 6: Final count of missing values
final_missing = players_df.isnull().sum().sum()
final_missing

np.int64(0)

In [156]:
# Check for missing values
print(coachs_df.isnull().sum().sum())

36


In [157]:
# Check for missing values
print(players_df.isnull().sum().sum())

0


In [158]:
# Check for missing values
print(doctors_df.isnull().sum().sum())

0


In [159]:
# Identify numerical columns
numeric_cols = all_leagues.select_dtypes(include=['int64', 'float64']).columns
from sklearn.preprocessing import MinMaxScaler

# Apply Min-Max Scaling
scaler = MinMaxScaler()
all_leagues[numeric_cols] = scaler.fit_transform(all_leagues[numeric_cols])

print("Min-Max Normalization Applied.")

Min-Max Normalization Applied.


In [160]:
# Identify numerical columns
numeric_cols = players_df.select_dtypes(include=['int64', 'float64']).columns
from sklearn.preprocessing import MinMaxScaler

# Apply Min-Max Scaling
scaler = MinMaxScaler()
players_df[numeric_cols] = scaler.fit_transform(players_df[numeric_cols])

In [161]:
from sklearn.ensemble import RandomForestRegressor

# Replace 'id' with 'ranking' in the dataset
all_leagues = all_leagues.rename(columns={'id': 'ranking'})

# Modeling


 📊 Club Performance Analyzer with Stakeholder Recommendations
 This model scores each club across key performance areas
 and provides advice on which stakeholder groups need the most improvement.


In [162]:
# 🎯 1. Define feature mapping for each position
position_features = {
    "GK": ["goals_conceded", "clean_sheets"],
    "CB": ["conceded_attacks_middle"],
    "RB": ["conceded_attacks_left"],
    "LB": ["conceded_attacks_right"],
    "DMF": ["conceded_attacks_middle", "possession(%)"],
    "CM": ["pass_accuracy (%)", "possession(%)"],
    "AMF": ["shots_per_match", "goals_scored"],
    "RW": ["total_attacks_left"],
    "LW": ["total_attacks_right"],
    "ST": ["goals_scored"],
    "Coach": ["wins", "draws", "losses", "goal_difference", "ranking"],
    "Medical Staff": ["total_injuries"]
    }

# 📦 2. Copy the normalized dataset
club_df = all_leagues.copy()


# 🧠 4. Score each position (average of normalized features × 100)
for position, features in position_features.items():
    club_df[f"{position}_Score"] = club_df[features].mean(axis=1) * 100

# 🤖 5. Recommendation logic – get 2 lowest scoring positions
def recommend_positions(row):
    scores = {pos: row[f"{pos}_Score"] for pos in position_features}
    weakest = sorted(scores.items(), key=lambda x: x[1])[:2]
    return f"⚠️ {row['club']} should improve " + " and ".join(pos for pos, _ in weakest)

club_df["Recommendations"] = club_df.apply(recommend_positions, axis=1)

# 📊 6. Display results
score_cols = [f"{pos}_Score" for pos in position_features]
result_df = club_df[["club"] + score_cols + ["Recommendations"]].sort_values(by="club")
result_df.head()

,club,GK_Score,CB_Score,RB_Score,LB_Score,DMF_Score,CM_Score,AMF_Score,RW_Score,LW_Score,ST_Score,Coach_Score,Medical Staff_Score,Recommendations
99,AC Milan,40.852459,85.714286,84.615385,84.615385,61.607143,50.677711,59.230382,70.0,72.727273,57.746479,45.897089,52.380952,⚠️ AC Milan should improve GK and Coach
101,AS Roma,42.754098,71.428571,69.230769,69.230769,54.464286,47.063253,42.823608,60.0,63.636364,38.028169,45.436235,38.095238,⚠️ AS Roma should improve ST and Medical Staff
21,AZ Alkmaar,36.295082,78.571429,76.923077,76.923077,58.035714,53.990964,65.618712,85.0,86.363636,63.380282,43.168151,52.380952,⚠️ AZ Alkmaar should improve GK and Coach
73,Ajaccio,50.262295,14.285714,15.384615,15.384615,10.642857,9.825301,5.952381,10.0,9.090909,0.000000,42.889609,14.285714,⚠️ Ajaccio should improve ST and AMF
20,Ajax,36.754098,92.857143,92.307692,92.307692,65.178571,64.533133,93.770959,100.0,100.000000,88.732394,45.691527,57.142857,⚠️ Ajax should improve GK and Coach


Create Model


In [163]:
import pandas as pd
import joblib

class PositionRecommender:
    def __init__(self):
        self.position_features = {
            "GK": ["goals_conceded", "clean_sheets"],
            "CB": ["conceded_attacks_middle"],
            "RB": ["conceded_attacks_left"],
            "LB": ["conceded_attacks_right"],
            "DMF": ["conceded_attacks_middle", "possession(%)"],
            "CM": ["pass_accuracy (%)", "possession(%)"],
            "AMF": ["shots_per_match", "goals_scored"],
            "RW": ["total_attacks_left"],
            "LW": ["total_attacks_right"],
            "ST": ["goals_scored"],
            "Coach": ["wins", "draws", "losses", "goal_difference", "ranking"],
            "Medical Staff": ["total_injuries"],
        }

    def score_positions(self, df: pd.DataFrame):
        df = df.copy()
        for position, features in self.position_features.items():
            df[f"{position}_Score"] = df[features].mean(axis=1) * 100
        return df

    def weakest_positions(self, df: pd.DataFrame):
        df = self.score_positions(df)

        def get_weakest(row):
            # Get the position scores and sort them
            scores = {pos: row[f"{pos}_Score"] for pos in self.position_features}
            # Get the top 5 weakest positions based on the lowest scores
            weakest = sorted(scores.items(), key=lambda x: x[1])[:5]
            return pd.Series({
                "club": row["club"],
                "Weakest_1": weakest[0][0],
                "Weakest_2": weakest[1][0],
                "Weakest_3": weakest[2][0],
                "Weakest_4": weakest[3][0],
                "Weakest_5": weakest[4][0]
            })

        # Apply the function to get the weakest positions for each club
        return df.apply(get_weakest, axis=1)

# Example data (replace this with your actual data)
# all_leagues = pd.read_csv("your_data.csv")  # Load the data as needed

# Create and use the model
model = PositionRecommender()
weakest_df = model.weakest_positions(all_leagues)

# Save the recommendations to CSV
weakest_df.to_csv("club_weakest_positions.csv", index=False)
print("📁 Weakest positions saved to 'club_weakest_positions.csv'")

# Save the model
joblib.dump(model, "position_recommender.pkl")
print("✅ Model saved to 'position_recommender.pkl'")


📁 Weakest positions saved to 'club_weakest_positions.csv'
✅ Model saved to 'position_recommender.pkl'


In [164]:
# Load and use the model
import joblib
recommender = joblib.load("position_recommender.pkl")

# Predict on all_leagues
result = recommender.weakest_positions(all_leagues)
result.head()


,club,Weakest_1,Weakest_2,Weakest_3,Weakest_4,Weakest_5
0,Bayern Munich,GK,Medical Staff,Coach,DMF,CM
1,Borussia Dortmund,Coach,GK,Medical Staff,CM,DMF
2,RB Leipzig,Coach,GK,Medical Staff,CM,ST
3,Union Berlin,CM,Medical Staff,ST,Coach,GK
4,SC Freiburg,ST,Coach,GK,Medical Staff,AMF


###The Second Model: Player Recommendation System


Step 1: Filter Players by Position and Nationality


Step 2: Score Players Based on Position-Specific Features
Now, the important part is scoring the players based on relevant features for their position. Here's how we will do that:

For Goalkeepers (GK), we will use features like gk_awareness, gk_catching, gk_reflexes, etc.

For Defenders (CB), we will use features like defensive_awareness, ball_winning, etc.

For Midfielders (DMF, CMF, etc.), we will use features like passing_accuracy, possession(%), etc.

For Attackers (ST, AMF, etc.), we will use features like goals_scored, shots_per_match, etc.

We will calculate a score for each player based on the selected position's features.

Step 3: Rank and Recommend the Top 5 Players


In [165]:
pip install fuzzywuzzy

In [166]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process

# 2. FootballRecommender class
class FootballRecommender:
    def __init__(self, players_file, transfers_file, clubs_file, coaches_file, doctors_file):
        self.players_df = pd.read_csv(players_file)
        self.transfers_df = pd.read_csv(transfers_file)
        self.clubs_df = pd.read_csv(clubs_file)
        self.coaches_df = pd.read_csv(coaches_file)
        self.doctors_df = pd.read_csv(doctors_file)

        self.formation_to_style = {
            '4-3-3 attacking': 'offensif', '3-4-2-1': 'offensif', '4-2-3-1': 'equilibre',
            '4-1-4-1': 'equilibre', '4-4-2 double 6': 'equilibre', '4-3-1-2': 'equilibre',
            '3-5-2 flat': 'equilibre', '4-4-2': 'equilibre', '5-3-2': 'defensif',
            '4-4-2 diamond': 'equilibre', '3-4-1-2': 'offensif'
        }

        print("\u2705 Fichiers chargés.")
        self.clean_data()
        self.build_club_profiles()

    def clean_data(self):
        self.players_df.dropna(subset=['team_name', 'league'], inplace=True)
        self.players_df['team_name'] = self.players_df['team_name'].str.lower().str.strip()
        self.players_df['name'] = self.players_df['name'].str.lower().str.strip()

        self.transfers_df.dropna(subset=['player_name', 'standardized_club'], inplace=True)
        self.transfers_df['standardized_club'] = self.transfers_df['standardized_club'].str.lower().str.strip()
        self.transfers_df['player_name'] = self.transfers_df['player_name'].str.lower().str.strip()

        self.clubs_df['club'] = self.clubs_df['club'].str.lower().str.strip()

        self.coaches_df_clean = self.coaches_df.dropna(subset=['preffered_formation', 'coaching_licence', 'club']).copy()
        self.coaches_df_clean['club'] = self.coaches_df_clean['club'].str.lower().str.strip()

        reference_clubs = self.clubs_df['club'].dropna().unique()
        def fuzzy_match_club(club_name, reference_clubs, threshold=80):
            match, score = process.extractOne(club_name, reference_clubs)
            return match if score >= threshold else None

        club_mapping = {club: fuzzy_match_club(club, reference_clubs) for club in self.coaches_df_clean['club'].unique()}
        self.coaches_df_clean['standardized_club'] = self.coaches_df_clean['club'].map(club_mapping)
        self.coaches_df_clean.dropna(subset=['standardized_club'], inplace=True)

        formation_encoder = LabelEncoder()
        licence_encoder = LabelEncoder()

        self.coaches_df_clean['formation_encoded'] = formation_encoder.fit_transform(self.coaches_df_clean['preffered_formation'])
        self.coaches_df_clean['licence_encoded'] = licence_encoder.fit_transform(self.coaches_df_clean['coaching_licence'])

        scaler = MinMaxScaler()
        self.coaches_df_clean['avg_term_as_coach'] = scaler.fit_transform(self.coaches_df_clean[['avg_term_as_coach']])

        self.coaches_df_clean['preffered_formation'] = self.coaches_df_clean['preffered_formation'].str.lower().str.strip()
        self.coaches_df_clean['style'] = self.coaches_df_clean['preffered_formation'].map(self.formation_to_style)

        self.player_features = [
            'age', 'speed', 'acceleration', 'dribbling', 'ball_control',
            'low_pass', 'finishing', 'defensive_awareness', 'physical_contact',
            'stamina', 'kicking_power', 'heading', 'tight_possession'
        ]
        self.player_features = [f for f in self.player_features if f in self.players_df.columns]

        self.players_df[self.player_features] = self.players_df[self.player_features].fillna(0)
        self.players_df[self.player_features] = scaler.fit_transform(self.players_df[self.player_features])

        print("\u2705 Données nettoyées.")

    def build_club_profiles(self):
        self.club_profiles = {}
        for club in self.transfers_df['standardized_club'].unique():
            club_players = self.transfers_df[self.transfers_df['standardized_club'] == club]['player_name'].unique()
            matched_players = self.players_df[self.players_df['name'].isin(club_players)]
            if not matched_players.empty:
                profile_vector = matched_players[self.player_features].mean(axis=0).values
                self.club_profiles[club] = profile_vector
        print(f"\u2705 {len(self.club_profiles)} profils de clubs créés.")

    def recommend_players_for_club(self, club_name, position=None, top_n=5):
        club_name = club_name.lower().strip()
        if club_name not in self.club_profiles:
            print(f"\u274c Aucun profil trouvé pour '{club_name}'.")
            return pd.DataFrame()

        club_vector = self.club_profiles[club_name].reshape(1, -1)
        temp_df = self.players_df.copy()

        if position:
            temp_df = temp_df[temp_df['registered_position'] == position.upper()]
            if temp_df.empty:
                print(f"\u274c Aucun joueur trouvé pour la position '{position}'.")
                return pd.DataFrame()

        similarities = cosine_similarity(temp_df[self.player_features], club_vector).flatten()
        temp_df['similarity_score'] = similarities

        return temp_df.sort_values(by='similarity_score', ascending=False).head(top_n)[
            ['name', 'team_name', 'league', 'registered_position', 'similarity_score']
        ]

    def recommend_coaches_for_club_by_style(self, club_name, top_n=5):
        club_name = club_name.lower().strip()
        temp_df = self.coaches_df_clean.copy()
        club_row = temp_df[temp_df['standardized_club'] == club_name]

        if club_row.empty:
            print(f"\u274c Aucun coach actuel trouvé pour '{club_name}'.")
            return pd.DataFrame()

        coach_formation = club_row.iloc[0]['preffered_formation']
        coach_style = self.formation_to_style.get(coach_formation, None)

        if coach_style is None:
            print(f"\u274c Formation '{coach_formation}' inconnue.")
            return pd.DataFrame()

        temp_df = temp_df[temp_df['style'] == coach_style]

        ideal_vector = np.array([[0.5, 0.5, temp_df['formation_encoded'].median(), temp_df['licence_encoded'].median()]])
        similarities = cosine_similarity(temp_df[['age', 'avg_term_as_coach', 'formation_encoded', 'licence_encoded']], ideal_vector).flatten()
        temp_df['similarity_score'] = similarities

        return temp_df.sort_values(by='similarity_score', ascending=False).head(top_n)[[
            'name', 'club', 'age', 'avg_term_as_coach', 'preffered_formation', 'coaching_licence', 'style', 'similarity_score'
        ]]

    def recommend_clubs_for_player(self, player_name, league_choice=None, top_n=5):
        player_name = player_name.lower().strip()
        player_row = self.players_df[self.players_df['name'] == player_name]

        if player_row.empty:
            print(f"\u274c Aucun joueur trouvé '{player_name}'.")
            return pd.DataFrame()

        player_vector = player_row[self.player_features].values.reshape(1, -1)

        similarities = {}
        for club, club_vector in self.club_profiles.items():
            sim = cosine_similarity(player_vector, club_vector.reshape(1, -1)).flatten()[0]
            similarities[club] = sim

        similarity_df = pd.DataFrame(list(similarities.items()), columns=['club', 'similarity_score'])
        return similarity_df.sort_values(by='similarity_score', ascending=False).head(top_n)

    def recommend_clubs_for_coach(self, coach_name, top_n=5):
        coach_name = coach_name.lower().strip()
        temp_df = self.coaches_df_clean.copy()
        coach_row = temp_df[temp_df['name'].str.lower().str.strip() == coach_name]

        if coach_row.empty:
            print(f"\u274c Aucun coach trouvé '{coach_name}'.")
            return pd.DataFrame()

        coach_formation = coach_row.iloc[0]['preffered_formation']
        coach_style = self.formation_to_style.get(coach_formation, None)

        if coach_style is None:
            print(f"\u274c Formation '{coach_formation}' inconnue.")
            return pd.DataFrame()

        clubs_with_style = temp_df[temp_df['style'] == coach_style]['standardized_club'].unique()
        clubs_df_filtered = self.clubs_df[self.clubs_df['club'].isin(clubs_with_style)].head(top_n)

        return clubs_df_filtered[['club', 'league', 'id', 'average_age']]

    def recommend_doctors_for_club(self, club_name, top_n=5):
        club_name = club_name.lower().strip()
        club_row = self.clubs_df[self.clubs_df['club'] == club_name]

        if club_row.empty:
            print(f"❌ Aucun club trouvé '{club_name}'.")
            return pd.DataFrame()

        club_total_injuries = club_row.iloc[0]['total_injuries']
        club_average_age = club_row.iloc[0]['average_age']

        temp_doctors = self.doctors_df.dropna(subset=['Tot_Benes', 'Bene_Avg_Age']).copy()

        temp_doctors['injury_diff'] = np.abs(temp_doctors['Tot_Benes'] - club_total_injuries)
        temp_doctors['age_diff'] = np.abs(temp_doctors['Bene_Avg_Age'] - club_average_age)

        temp_doctors['injury_diff'] = temp_doctors['injury_diff'] / (temp_doctors['injury_diff'].max() + 1e-6)
        temp_doctors['age_diff'] = temp_doctors['age_diff'] / (temp_doctors['age_diff'].max() + 1e-6)

        temp_doctors['similarity_score'] = 1 - (temp_doctors['injury_diff'] + temp_doctors['age_diff']) / 2

        return temp_doctors.sort_values(by='similarity_score', ascending=False).head(top_n)[[
            'Rndrng_Prvdr_First_Name', 'Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_Type', 'similarity_score'
        ]]


    def recommend_clubs_for_doctor(self, doctor_name, search_by='last_name', top_n=5):
        doctor_name = doctor_name.lower().strip()
        temp_doctors = self.doctors_df.copy()

        if search_by == 'first_name':
            temp_doctors['search_field'] = temp_doctors['Rndrng_Prvdr_First_Name'].fillna('').str.lower().str.strip()
        else:
            temp_doctors['search_field'] = temp_doctors['Rndrng_Prvdr_Last_Org_Name'].fillna('').str.lower().str.strip()

        doctor_row = temp_doctors[temp_doctors['search_field'] == doctor_name]

        if doctor_row.empty:
            print(f"❌ Aucun docteur trouvé '{doctor_name}'.")
            return pd.DataFrame()

        doctor_patients = doctor_row.iloc[0]['Tot_Benes']
        doctor_avg_age = doctor_row.iloc[0]['Bene_Avg_Age']

        temp_clubs = self.clubs_df.dropna(subset=['total_injuries', 'average_age']).copy()

        temp_clubs['injury_diff'] = np.abs(temp_clubs['total_injuries'] - doctor_patients)
        temp_clubs['age_diff'] = np.abs(temp_clubs['average_age'] - doctor_avg_age)

        temp_clubs['injury_diff'] = temp_clubs['injury_diff'] / (temp_clubs['injury_diff'].max() + 1e-6)
        temp_clubs['age_diff'] = temp_clubs['age_diff'] / (temp_clubs['age_diff'].max() + 1e-6)

        temp_clubs['similarity_score'] = 1 - (temp_clubs['injury_diff'] + temp_clubs['age_diff']) / 2

        return temp_clubs.sort_values(by='similarity_score', ascending=False).head(top_n)[[
            'club', 'league', 'average_age', 'similarity_score'
        ]]

# ⚡ DO NOT IMPORT AGAIN because FootballRecommender is already loaded above!

# 1. Initialize the recommender
print("\n⚡ Initialisation du modèle...")
recommender = FootballRecommender(
    players_file='playerss.csv',
    transfers_file='/content/Transfers_with_Standardized_Team_Names .csv',
    clubs_file='clubs_n.csv',
    coaches_file='/content/cleaned_coaches (1).csv',
    doctors_file='/content/doctorsRa.csv'
)

# 2. Test the different recommendation functions

print("\n🎯 Recommandation de joueurs pour Ajax (Position: GK) :")
players_recommended = recommender.recommend_players_for_club('ajax', position='GK', top_n=5)
print(players_recommended)

print("\n🎯 Recommandation de coachs pour Juventus :")
coaches_recommended = recommender.recommend_coaches_for_club_by_style('juventus', top_n=5)
print(coaches_recommended)

print("\n🎯 Recommandation de clubs pour L. Messi (League: La Liga) :")
clubs_for_player = recommender.recommend_clubs_for_player('l. messi', league_choice='La Liga', top_n=5)
print(clubs_for_player)

print("\n🎯 Recommandation de clubs pour Pep Guardiola :")
clubs_for_coach = recommender.recommend_clubs_for_coach('pep guardiola', top_n=5)
print(clubs_for_coach)

print("\n🎯 Recommandation de docteurs pour Ajax :")
doctors_for_club = recommender.recommend_doctors_for_club('ajax', top_n=5)
print(doctors_for_club)

print("\n🎯 Recommandation de clubs pour un docteur (exemple avec 'homan') :")
clubs_for_doctor = recommender.recommend_clubs_for_doctor('homan', search_by='last_name', top_n=5)
print(clubs_for_doctor)

print("\n🎯 Recommandation de clubs pour un docteur (exemple avec 'zena' comme prénom) :")
clubs_for_doctor_by_firstname = recommender.recommend_clubs_for_doctor('zena', search_by='first_name', top_n=5)
print(clubs_for_doctor_by_firstname)




⚡ Initialisation du modèle...
✅ Fichiers chargés.
✅ Données nettoyées.
✅ 103 profils de clubs créés.

🎯 Recommandation de joueurs pour Ajax (Position: GK) :
                  name            team_name                    league  \
8332   wellington luís          free agents                Free Agent   
11269    claudio vitor                 avaí  BRASILEIRÃO SÉRIE B 2020   
14495           g. ito          vissel kobe      AFC Champions League   
6961       park il-gyu  yokohama f. marinos      AFC Champions League   
7646       nuno macedo          free agents                Free Agent   

      registered_position  similarity_score  
8332                   GK          0.976212  
11269                  GK          0.969936  
14495                  GK          0.965162  
6961                   GK          0.959809  
7646                   GK          0.956979  

🎯 Recommandation de coachs pour Juventus :
               name           club   age  avg_term_as_coach  \
36     Thiago Motta 

In [167]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widgets principaux
recommendation_type = widgets.Dropdown(
    options=[
        'Recommander joueurs/coachs/doctors pour un club',
        'Recommander clubs pour un joueur/coach/doctor'
    ],
    value='Recommander joueurs/coachs/doctors pour un club',
    description='Type:',
)

role_dropdown = widgets.Dropdown(
    options=['player', 'coach', 'doctor'],
    value='player',
    description='Rôle:',
)

club_or_name_text = widgets.Text(
    value='',
    description='Nom:',
    placeholder='Ex: Ajax ou Messi ou John Doe'
)

position_text = widgets.Text(
    value='',
    description='Position:',
    placeholder='Ex: GK'
)

submit_button = widgets.Button(
    description='🎯 Recommander',
    button_style='success'
)

output = widgets.Output()

# Fonction exécutée au clic
def on_submit_button_clicked(b):
    with output:
        clear_output()

        type_reco = recommendation_type.value
        role = role_dropdown.value
        name = club_or_name_text.value.strip()
        position = position_text.value.strip()

        if not name:
            print("⚠️ Veuillez entrer un nom.")
            return

        if type_reco == 'Recommander joueurs/coachs/doctors pour un club':
            if role == 'player':
                if not position:
                    print("⚠️ Veuillez entrer une position pour le joueur.")
                    return
                print(f"🎯 Recommandation de joueurs pour {name} (Position: {position}) :")
                result = recommender.recommend_players_for_club(name, position=position, top_n=5)
                display(result)
            elif role == 'coach':
                print(f"🎯 Recommandation de coachs pour {name} :")
                result = recommender.recommend_coaches_for_club_by_style(name, top_n=5)
                display(result)
            elif role == 'doctor':
                print(f"🎯 Recommandation de docteurs pour {name} :")
                result = recommender.recommend_doctors_for_club(name, top_n=5)
                display(result)

        elif type_reco == 'Recommander clubs pour un joueur/coach/doctor':
            if role == 'player':
                print(f"🎯 Recommandation de clubs pour le joueur {name} :")
                result = recommender.recommend_clubs_for_player(name, league_choice=None, top_5_only=False, top_n=5)
                display(result)
            elif role == 'coach':
                print(f"🎯 Recommandation de clubs pour le coach {name} :")
                result = recommender.recommend_clubs_for_coach(name, top_n=5)
                display(result)
            elif role == 'doctor':
                print(f"🎯 Recommandation de clubs pour le docteur {name} :")
                result = recommender.recommend_clubs_for_doctor(name, top_n=5)
                display(result)

submit_button.on_click(on_submit_button_clicked)

# Contrôle dynamique d'affichage
def update_fields_visibility(change):
    if recommendation_type.value == 'Recommander joueurs/coachs/doctors pour un club':
        club_or_name_text.description = 'Club:'
        if role_dropdown.value == 'player':
            position_text.layout.display = 'block'
        else:
            position_text.layout.display = 'none'
    else:
        club_or_name_text.description = 'Nom:'
        position_text.layout.display = 'none'

recommendation_type.observe(update_fields_visibility, names='value')
role_dropdown.observe(update_fields_visibility, names='value')
update_fields_visibility(None)

# Affichage
display(recommendation_type, role_dropdown, club_or_name_text, position_text, submit_button, output)


Dropdown(description='Type:', options=('Recommander joueurs/coachs/doctors pour un club', 'Recommander clubs p…

Dropdown(description='Rôle:', options=('player', 'coach', 'doctor'), value='player')

Text(value='', description='Club:', placeholder='Ex: Ajax ou Messi ou John Doe')

Text(value='', description='Position:', layout=Layout(display='block'), placeholder='Ex: GK')

Button(button_style='success', description='🎯 Recommander', style=ButtonStyle())

Output()